In [2]:
import numpy as np
import scipy
import scipy.sparse.linalg as spla
from PIL import Image
import os
import time
import sys
from log_utils import log
import numba as nb
from numba import njit
# import pytensor
import cupy as cp
# import cusolver
import cupyx.scipy.sparse.linalg as cupy_spla

In [7]:
pwd

'/mnt/c/myProject/phd/what/software/fill_depth_colorization/src'

In [9]:
use_gpu = True

def get_imlist(path):
    return [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.png')]

sparse_path = "/mnt/c/myProject/phd/what/software/data/dataset/ssl/cepton/cepton_1/val/sparse_depth/"
output_folder = "../results/ssl/cepton/cepton_1/"
# eval_output_path = sys.argv[3]
# log_path = os.path.join(eval_output_path, 'results.txt')

# test_path = '/mnt/c/myProject/phd/what/software/data/dataset/kitti_test/val/image/0000000005.png'

sparse_list = get_imlist(sparse_path)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def colorisation(imgDepth=None, kernel='cross'): # kernel= 'cross' or 'full', 'cross uses 4 nearest neighbours, 'full' uses 8 nearest neighbours
    imgNotValid = imgDepth == 0 # if pixel value is 0, then it is True, else False
    maxImgAbsDepth = np.max(imgDepth)
    imgDepth[imgDepth > maxImgAbsDepth] = maxImgAbsDepth
    # imgDepth = imgDepthInput / maxImgAbsDepth
    # imgDepth[imgDepth > 1] = 1
    H, W = imgDepth.shape
    numPix = H * W
    indsM = np.arange(numPix).reshape((W, H)).T # [[0 375 750 ... 467625 468000 468375][1 376 751 ... 467626 468001 468376]...[374 749 1124 ... 467999 468374 468749]]
    knownValMask = (~imgNotValid).astype(int) # 0 when the pixel value is 0 (True), 1 when the pixel value is not 0 (False)
    winRad = 1  # default window radius = 1
    # Initialize variables
    len_ = 0
    absImgNdx = 0
    # the windows determine how far the neighbouring pixels affecting the center pixel
    len_window = (winRad * 4) + 1  # cross kernel
    len_zeros = numPix * len_window

    # Original loop
    # array with length numPix * len_window, 468750 *len_window
    cols = np.zeros(len_zeros, dtype=int)
    rows = np.zeros(len_zeros, dtype=int)
    vals = np.zeros(len_zeros, dtype=float)
    gvals = np.zeros(len_window, dtype=float)

    # loop for all pixels as the center pixel i,j
    for j in range(W):
        for i in range(H):
            nWin = 0
            # loop for all neighbouring pixels affecting the center pixel depending on your window radius
            for ii in range(max(0, i - winRad), min(i + winRad + 1, H)):
                for jj in range(max(0, j - winRad), min(j + winRad + 1, W)):
                    # if it is the center pixel, continue to next jj
                    if ii == i and jj == j:
                        continue
                    if kernel == 'cross':
                        if abs(ii - i) > 0 and abs(jj - j) > 0:
                            continue

                    rows[len_] = absImgNdx # store index from 0 to H*W, e.g. H = 375,W=1250, index is from 0 to 468750, [0,0,1,1,1,,2,2,2,...,468750,468750]
                    cols[len_] = indsM[ii, jj] # store the value at [ii,jj], values are indices, indices from 0 to H*W, e.g. H = 375,W=1250, index is from 0 to 468750, [375 1 376 , 0 375 376 ,2,377,3,...]
                    # print([ii,jj])
                    # indsM
                    # [[     0    375    750 ... 467625 468000 468375]
                    #  [     1    376    751 ... 467626 468001 468376]
                    #  [     2    377    752 ... 467627 468002 468377]
                    #  ...
                    #  [   372    747   1122 ... 467997 468372 468747]
                    #  [   373    748   1123 ... 467998 468373 468748]
                    #  [   374    749   1124 ... 467999 468374 468749]]

                    # For row, first pixel index 0, it has 2 neighbours,second pixel index 1 has 3 neighbours, third pixel 2 has 3 neighbours
                    # rows[len_] = absImgNdx # store index from 0 to H*W, e.g. H = 375,W=1250, index is from 0 to 468750, [0,0,1,1,1,,2,2,2,...,468750,468750]
                    # For column index 0 has neighbours 375,1, index 1 has neighbours 0,375,376,2,377
                    # cols[len_] = indsM[ii, jj] # store the value at [ii,jj], values are indices, indices from 0 to H*W, e.g. H = 375,W=1250, index is from 0 to 468750, [375 1 , 0 376 2 ,...]
                    # gval[] for the first pixel is [ 0.  0.  0.  0. -1. -1. -1. -1. -1.]

                    len_ = len_ + 1
                    nWin = nWin + 1

            # equal weighting for all neighbours
            gvals[:nWin] = 1
            gvals[:nWin] /= np.sum(gvals[:nWin])

            # # random weights
            # # # Calculate the number of elements in the array
            # # nWin = len(gvals)
            # # Generate random weights
            # weights = np.random.rand(nWin)
            # # Normalize the weights
            # weights /= np.sum(weights)
            # # Assign the random weights to the gvals array
            # gvals[:nWin] = weights
            # print(gvals)
            # print(ok)

            vals[len_ - nWin:len_] = -gvals[:nWin]

            # Now the self-reference (along the diagonal).
            rows[len_] = absImgNdx # index for the center pixel , len_ = 3,9,15,...
            cols[len_] = absImgNdx 
            vals[len_] = 1  # sum(gvals(1:nWin))

            len_ += 1
            absImgNdx += 1


    rows = rows[:len_] # rows = [0.     0.    0.   1.   1.     1.   1.   2.    2.      2.      2.   3. 3. 3. 3.  ...], for pixel 0, it has 2 neighbours and itself
    cols = cols[:len_] # cols = [375.   1.    0.   0.   376.   2.   1.   1.    377.    3.      2.                           ...]
    vals = vals[:len_] # vals = [-0.5 -0.5    1.  -0.33 -0.33 -0.33 1   -0.33 -0.33 -0.33 1.  -0.33 -0.33 -0.33 1.                          ...]
    A = scipy.sparse.csr_matrix((vals, (rows, cols)), (numPix, numPix)) # should be a large sparse matrix with diagonals are the blocks containing the neighbouring pixels

    rows = np.arange(0, numPix) # rows = [0      1     2     3    4    5    6     7    8     9    10    11   12    13                           ...]
    cols = np.arange(0, numPix) # cols = [0      1     2     3    4    5    6     7    8     9    10    11   12    13                           ...]
    vals = knownValMask.T.reshape(numPix) # values are mostly zero on the pixels that no sparse point cloud and 1 on valid pixels
    G = scipy.sparse.csr_matrix((vals, (rows, cols)), (numPix, numPix))

    A += G
    b = np.multiply(vals, imgDepth.flatten('F'))

    new_vals = spla.spsolve(A, b)
    output = np.reshape(new_vals, (H, W), 'F')

    return output

# create matrix using gpu
@nb.njit
def matrix_gpu(imgDepth=None, kernel='cross'): # kernel= 'cross' or 'full', 'cross uses 4 nearest neighbours, 'full' uses 8 nearest neighbours
    H, W = imgDepth.shape
    numPix = H * W
    indsM = np.arange(numPix).reshape((W, H)).T # [[0 375 750 ... 467625 468000 468375][1 376 751 ... 467626 468001 468376]...[374 749 1124 ... 467999 468374 468749]]
    winRad = 1  # default window radius = 1
    # Initialize variables
    len_ = 0
    absImgNdx = 0
    # the windows determine how far the neighbouring pixels affecting the center pixel
    len_window = (winRad * 4) + 1  # cross kernel
    len_zeros = numPix * len_window

    # Original loop
    # array with length numPix * len_window, 468750 *len_window, with values = -1
    cols = np.zeros(len_zeros, dtype=np.int64)  # Specify dtype as np.int64
    rows = np.zeros(len_zeros, dtype=np.int64)
    len_rows = len(rows)
    vals = np.zeros(len_rows, dtype=np.float32)
    gvals = np.zeros(len_window, dtype=np.float32)

    # loop for all pixels as the center pixel i,j
    for j in range(W):
        for i in range(H):
            nWin = 0
            # loop for all neighbouring pixels affecting the center pixel depending on your window radius
            for ii in range(max(0, i - winRad), min(i + winRad + 1, H)):
                for jj in range(max(0, j - winRad), min(j + winRad + 1, W)):
                    # if it is the center pixel, continue to next jj
                    if ii == i and jj == j:
                        continue
                    if kernel == 'cross':
                        if abs(ii - i) > 0 and abs(jj - j) > 0:
                            continue

                    rows[len_] = absImgNdx # store index from 0 to H*W, e.g. H = 375,W=1250, index is from 0 to 468750, [0,0,1,1,1,,2,2,2,...,468750,468750]
                    cols[len_] = indsM[ii, jj] # store the value at [ii,jj], values are indices, indices from 0 to H*W, e.g. H = 375,W=1250, index is from 0 to 468750, [375 1 376 , 0 375 376 ,2,377,3,...]
                    # print([ii,jj])
                    # indsM
                    # [[     0    375    750 ... 467625 468000 468375]
                    #  [     1    376    751 ... 467626 468001 468376]
                    #  [     2    377    752 ... 467627 468002 468377]
                    #  ...
                    #  [   372    747   1122 ... 467997 468372 468747]
                    #  [   373    748   1123 ... 467998 468373 468748]
                    #  [   374    749   1124 ... 467999 468374 468749]]

                    # For row, first pixel index 0, it has 2 neighbours,second pixel index 1 has 3 neighbours, third pixel 2 has 3 neighbours
                    # rows[len_] = absImgNdx # store index from 0 to H*W, e.g. H = 375,W=1250, index is from 0 to 468750, [0,0,1,1,1,,2,2,2,...,468750,468750]
                    # For column index 0 has neighbours 375,1, index 1 has neighbours 0,375,376,2,377
                    # cols[len_] = indsM[ii, jj] # store the value at [ii,jj], values are indices, indices from 0 to H*W, e.g. H = 375,W=1250, index is from 0 to 468750, [375 1 , 0 376 2 ,...]
                    # gval[] for the first pixel is [ 0.  0.  0.  0. -1. -1. -1. -1. -1.]

                    len_ = len_ + 1
                    nWin = nWin + 1

            # equal weighting for all neighbours
            gvals[:nWin] = 1
            gvals[:nWin] /= np.sum(gvals[:nWin])

            vals[len_ - nWin:len_] = -gvals[:nWin]

            # Now the self-reference (along the diagonal).
            rows[len_] = absImgNdx # index for the center pixel , len_ = 3,9,15,...
            cols[len_] = absImgNdx 
            vals[len_] = 1  # sum(gvals(1:nWin))

            len_ += 1
            absImgNdx += 1
    

    rows_A = rows[:len_] # rows = [0.     0.    0.   1.   1.     1.   1.   2.    2.      2.      2.   3. 3. 3. 3.  ...], for pixel 0, it has 2 neighbours and itself
    cols_A = cols[:len_] # cols = [375.   1.    0.   0.   376.   2.   1.   1.    377.    3.      2.                           ...]
    vals_A = vals[:len_] # vals = [-0.5 -0.5    1.  -0.33 -0.33 -0.33 1   -0.33 -0.33 -0.33 1.  -0.33 -0.33 -0.33 1.                          ...]

    return rows_A, cols_A ,vals_A

def depth_read(filename):
    # loads depth map D from png file
    # and returns it as a numpy array
    depth = np.array(Image.open(filename), dtype=int)

    return depth

num_sample = len(sparse_list)
print("number of samples: ", num_sample)
time_elapse = 0
print("start")

for sparse in sparse_list[0:1]: # test for first image
# for sparse in sparse_list:
    output_name = os.path.basename(sparse).split('/')[-1]
    output_path = os.path.join(output_folder, output_name)
    sparse = depth_read(sparse)
    if len(sparse.shape) == 3:
        sparse = sparse[:, :, 0]
    
    start_time = time.time()

    if use_gpu == True:
        imgNotValid = sparse == 0
        maxImgAbsDepth = np.max(sparse)
        sparse = np.where(sparse > maxImgAbsDepth, maxImgAbsDepth, sparse)
        knownValMask = (~imgNotValid).astype(int)
        H, W = sparse.shape
        numPix = H * W

        # rows_A, cols_A ,vals_A = matrix_gpu(sparse, kernel='cross')
        # rows_G = np.arange(0, numPix)
        # cols_G = np.arange(0, numPix)
        # vals_G = knownValMask.T.reshape(numPix)

        # A = scipy.sparse.csr_matrix((vals_A, (rows_A, cols_A)), (numPix, numPix))  
        # G = scipy.sparse.csr_matrix((vals_G, (rows_G, cols_G)), (numPix, numPix))
        # A += G
        # b = np.multiply(vals_G, sparse.flatten('F'))

        # -----------------------------------------------------------------------------------#
        # # use numba to create matrix and scipy to solve the equation
        # new_vals = spla.spsolve(A, b) # Solve the sparse linear system Ax=b
        # # new_vals, exit_code = spla.bicgstab(A, b) # Use BIConjugate Gradient STABilized iteration to solve Ax = b
        # # print("exit_code: ", exit_code)  # 0 indicates successful convergence
        # result = np.reshape(new_vals, (H, W), 'F')

        # -----------------------------------------------------------------------------------#
        # # use numba to create matrix and pytensor to solve the equation but has error in pytensor

        # new_vals = pytensor.tensor.slinalg.solve(A, b)
        # result = np.reshape(new_vals, (H, W), 'F')

        # -----------------------------------------------------------------------------------#
        # # use numba to create matrix and cuSOLVER to solve the equation, still error for cupyx
 
        # # Convert to CuPy-compatible format
        # A = csp.coo_matrix(A)
        # b = np.multiply(vals_G, sparse.flatten('F'))
        # # Convert to CuPy-compatible format
        # b = cp.array(b)

        # # Create a cuSOLVER handle
        # handle = cusolver.create()
        # # Get the required parameters
        # num_rows = A.shape[0]
        # nnz = A.nnz
        # A_val = A.data
        # A_row = A.row
        # A_col = A.col
        # tol = 1e-6 # small positive value, such as 1e-6 or 1e-8

        # new_vals = np.empty(numPix, dtype=np.float64)  # solution vector
        # cusolver.cusolverSpDcsrlsqvqr(handle, num_rows, nnz, A_val, A_row, A_col, b, tol, new_vals)

        # result = np.reshape(new_vals, (H, W), 'F')

        # # Destroy the cuSOLVER handle
        # cusolver.destroy(handle)

        # -----------------------------------------------------------------------------------#
        # # use numba to create matrix and cupy to solve the equation, but has errors in cuda
        rows_A, cols_A, vals_A = matrix_gpu(sparse, kernel='cross')
        rows_G = cp.arange(0, numPix)
        cols_G = cp.arange(0, numPix)
        vals_G_np = knownValMask.T.reshape(numPix)
        
        # Create the sparse matrix
        rows_A = cp.asarray(rows_A, dtype=cp.float32)
        cols_A = cp.asarray(cols_A, dtype=cp.float32)
        vals_A = cp.asarray(vals_A, dtype=cp.float32)
        vals_G = cp.asarray(vals_G_np, dtype=cp.float32)

        A = cp.sparse.csr_matrix((vals_A, (rows_A, cols_A)), (numPix, numPix))  
        G = cp.sparse.csr_matrix((vals_G, (rows_G, cols_G)), (numPix, numPix))
        A += G
        # sparse = cp.asarray(sparse, dtype=cp.float32)
        # b = cp.multiply(vals_G, sparse.flatten('F'))
        b = np.multiply(vals_G_np, sparse.flatten('F'))
        b = cp.asarray(b, dtype=cp.float32)
        
        new_vals = cupy_spla.spsolve(A, b)
        result = np.reshape(new_vals, (H, W), 'F')
        # -----------------------------------------------------------------------------------#
        # # Create a sparse matrix # example to test
        # data = cp.array([1, 2, 3], dtype=cp.float32)
        # row = cp.array([0, 1, 2], dtype=cp.int32)
        # col = cp.array([0, 1, 2], dtype=cp.int32)
        # num_rows = 3
        # num_cols = 3
        # A = cp.sparse.csr_matrix((data, (row, col)), shape=(num_rows, num_cols))

        # # Create the right-hand side vector
        # b = cp.array([1, 2, 3], dtype=cp.float32)

        # # Solve the sparse linear system
        # x = cupy_spla.spsolve(A, b)

        # # Convert the solution back to a NumPy array if needed
        # x = cp.asnumpy(x)
        # print(x)
        # print(ok)
    else:
        result = colorisation(sparse, kernel='cross')

    time_elapse +=  time.time() - start_time
    print("--- %s seconds ---" % (time.time() - start_time))
print("finished")

#     if np.amax(result) < 256:
#         result = np.round((result / 255) * 65535).astype(np.uint16)
#     result = np.uint32(result)
#     result = Image.fromarray(result, mode='I') 
#     result.save(output_path) # result is 0 to 65535

# time_elapse_per_sample = time_elapse/float(num_sample)
# log('Total testing time: {:.2f} min  Average testing time per sample: {:.2f} ms'.format(
#     time_elapse / 60.0, time_elapse_per_sample * 1000.0),
#     log_path)

number of samples:  33
start


CUSOLVERError: CUSOLVER_STATUS_ALLOC_FAILED